### LSTM model for anomaly detection

In [ ]:
This notebook cotains the LSTM model used along with autoencoder to detect anomalies in this case being ECD fromot

In [ ]:
import pandas as pd

In [ ]:
udata = pd.read_csv("/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/Windowed Time Series/un_cal.csv")
pdata = pd.read_csv("/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/Windowed Time Series/ECD_cal.csv")

In [ ]:
pdata['Label'] = "ECD"

In [ ]:
udata['Label'] = "Unsuccessful"

In [ ]:
data = pd.concat([udata,pdata])

In [ ]:
data['Label'].value_counts()

In [ ]:
### we will consider all unsuccessful waveforms for training the model.

In [ ]:
un_data = udata.iloc[:,1:-1]

In [ ]:
un_data.shape

In [ ]:
ECD_data = pdata.iloc[:,1:-1]

In [ ]:
ECD_data.shape

In [ ]:
ECD_ts = pd.read_csv('/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/PC_TS_STAND.csv')
un_ts = pd.read_csv('/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/US_TS_STAND.csv')
sy_ts = pd.read_csv('/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/PC_TS_SYNTH_STAND.csv')

In [ ]:
un_ts['Label'] = "Unsuccessful"
ECD_ts['Label'] ="ECD"
sy_ts['Label'] ="ECD"

In [ ]:
ECD_pos = pd.concat([ECD_ts,sy_ts])

In [ ]:
un_data = un_ts.iloc[:,1:-1]

In [ ]:
un_data.shape

In [ ]:
ECD_data = ECD_pos.iloc[:,1:-1]

In [ ]:
ECD_data.shape

## We’ll split the unsuccessful examples into train, validation and test sets:

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch

RANDOM_SEED = 123

DEVICE = ('cuda:0' if torch.cuda.is_available() else 'cpu')

train_df, val_df = train_test_split(
  un_data,
  test_size=0.15,
  random_state=RANDOM_SEED
)
val_df, test_df = train_test_split(
  val_df,
  test_size=0.33,
  random_state=RANDOM_SEED
)

In [ ]:
!pip install torch

In [ ]:
def create_dataset(df):
    sequences = df.astype(np.float32).to_numpy().tolist()
    dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]
    n_seq, seq_len, n_features = torch.stack(dataset).shape
    return dataset, seq_len, n_features

In [ ]:
train_dataset, seq_len, n_features = create_dataset(train_df)
val_dataset, _, _ = create_dataset(val_df)
test_normal_dataset, _, _ = create_dataset(test_df)
test_anomaly_dataset, _, _ = create_dataset(ECD_data)

In [ ]:
import torch.nn as nn

In [ ]:
class Encoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(Encoder, self).__init__()
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
        self.rnn1 = nn.LSTM(
            input_size=n_features,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.rnn2 = nn.LSTM(
            input_size=self.hidden_dim,
            hidden_size=embedding_dim,
            num_layers=1,
            batch_first=True
        )
    def forward(self, x):
        x = x.reshape((1, self.seq_len, self.n_features))
        x, (_, _) = self.rnn1(x)
        x, (hidden_n, _) = self.rnn2(x)
        return hidden_n.reshape((self.n_features, self.embedding_dim))

In [ ]:
class Decoder(nn.Module):
    def __init__(self, seq_len, input_dim=64, n_features=1):
        super(Decoder, self).__init__()
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 2 * input_dim, n_features
        self.rnn1 = nn.LSTM(
            input_size=input_dim,
            hidden_size=input_dim,
            num_layers=1,
            batch_first=True
        )
        self.rnn2 = nn.LSTM(
            input_size=input_dim,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.output_layer = nn.Linear(self.hidden_dim, n_features)
    def forward(self, x):
        x = x.repeat(self.seq_len, self.n_features)
        x = x.reshape((self.n_features, self.seq_len, self.input_dim))
        x, (hidden_n, cell_n) = self.rnn1(x)
        x, (hidden_n, cell_n) = self.rnn2(x)
        x = x.reshape((self.seq_len, self.hidden_dim))
        return self.output_layer(x)

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(seq_len, n_features, embedding_dim).to(DEVICE)
        self.decoder = Decoder(seq_len, embedding_dim, n_features).to(DEVICE)
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
model = Autoencoder(seq_len, n_features, 128)
model = model.to(DEVICE)

In [ ]:
import copy

def train_model(model, train_dataset, val_dataset, n_epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.L1Loss(reduction='sum').to(DEVICE)
    history = dict(train=[], val=[])
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000.0
    for epoch in range(1, n_epochs + 1):
        model = model.train()
        train_losses = []
        for seq_true in train_dataset:
                optimizer.zero_grad()
                seq_true = seq_true.to(DEVICE)
                seq_pred = model(seq_true)
                loss = criterion(seq_pred, seq_true)
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())
        val_losses = []
        model = model.eval()
        with torch.no_grad():
            for seq_true in val_dataset:
                seq_true = seq_true.to(DEVICE)
                seq_pred = model(seq_true)
                loss = criterion(seq_pred, seq_true)
                val_losses.append(loss.item())
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
    model.load_state_dict(best_model_wts)
    return model.eval(), history

In [ ]:
model, history = train_model(
  model,
  train_dataset,
  val_dataset,
  n_epochs=10
)

In [ ]:
def predict(model, dataset):
    predictions, losses = [], []
    criterion = nn.L1Loss(reduction='sum').to(DEVICE)
    with torch.no_grad():
        model = model.eval()
        for seq_true in dataset:
            seq_true = seq_true.to(DEVICE)
            seq_pred = model(seq_true)
            loss = criterion(seq_pred, seq_true)
            predictions.append(seq_pred.cpu().numpy().flatten())
            losses.append(loss.item())
        return predictions, losses

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns

_, losses = predict(model, train_dataset)
sns.histplot(losses, bins=50, kde=True);

In [ ]:
THRESHOLD = 10

In [ ]:
predictions, pred_losses = predict(model, test_normal_dataset)
sns.histplot(pred_losses, bins=50, kde=True);

In [ ]:
correct = sum(l <= THRESHOLD for l in pred_losses)
print(f'Correct normal predictions: {correct}/{len(test_normal_dataset)}')


In [ ]:
predictions, pred_losses = predict(model, test_anomaly_dataset)
sns.histplot(pred_losses, bins=50, kde=True);

In [ ]:
correct = sum(l > THRESHOLD for l in pred_losses)
print(f'Correct anomaly predictions: {correct}/{len(test_anomaly_dataset)}')